# Load DB

In [1]:
import sqlite3
conn = sqlite3.connect('morphology.db')
c = conn.cursor()

# Get all the words

In [9]:
c.execute("""
CREATE TABLE words (word text)
""")

In [11]:
import pandas as pd
import requests
import itertools

letters = 'AĀBCČDEĒFGĢHIĪJKĶLĻMNŅOŌPQRSŠTUŪVZ'

for letter in letters:
    for page in itertools.count(1):
        print(letter, page)
        df = pd.read_html(requests.get(
            f"https://www.letonika.lv/groups/getEntryList.aspx?q={letter}&page={page}"
            "&pos=0&resID=10621033&sid=0.9246002265161375"
        ).text)
        c.executemany(
            "INSERT INTO words VALUES (?)",
            df[0].values
        )
        if "nākamie šķirkļi" not in df[0].values:
            break

A 1
A 2
A 3
A 4
A 5
A 6
A 7
A 8
A 9
A 10
A 11
A 12
A 13
A 14
A 15
A 16
A 17
A 18
A 19
A 20
A 21
A 22
A 23
A 24
A 25
A 26
A 27
A 28
A 29
Ā 1
Ā 2
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
C 1
C 2
C 3
C 4
C 5
Č 1
Č 2
D 1
D 2
D 3
D 4
D 5
D 6
D 7
D 8
D 9
D 10
D 11
D 12
D 13
D 14
D 15
D 16
D 17
E 1
E 2
E 3
E 4
E 5
E 6
E 7
Ē 1
F 1
F 2
F 3
F 4
F 5
F 6
G 1
G 2
G 3
G 4
G 5
G 6
G 7
G 8
Ģ 1
H 1
H 2
H 3
I 1
I 2
I 3
I 4
I 5
I 6
I 7
I 8
I 9
I 10
I 11
I 12
I 13
I 14
I 15
I 16
I 17
I 18
I 19
I 20
I 21
I 22
Ī 1
J 1
J 2
J 3
K 1
K 2
K 3
K 4
K 5
K 6
K 7
K 8
K 9
K 10
K 11
K 12
K 13
K 14
K 15
K 16
K 17
K 18
K 19
K 20
K 21
K 22
Ķ 1
L 1
L 2
L 3
L 4
L 5
L 6
L 7
L 8
L 9
L 10
L 11
Ļ 1
M 1
M 2
M 3
M 4
M 5
M 6
M 7
M 8
M 9
M 10
M 11
M 12
M 13
M 14
M 15
M 16
N 1
N 2
N 3
N 4
N 5
N 6
N 7
N 8
N 9
N 10
N 11
N 12
N 13
N 14
N 15
N 16
N 17
N 18
N 19
N 20
N 21
Ņ 1
O 1
O 2
O 3
Ō 1
P 1
P 2
P 3
P 4
P 5
P 6
P 7
P 8
P 9
P 10
P 11
P 12
P 13
P 14
P 15
P 16
P 17
P 18
P 19
P 20
P 21
P 22
P 23
P 24
P 25
P 26
P 27
P 28
P 29
P 

In [12]:
conn.commit()

# Webpage

In [113]:
c.execute("""
CREATE TABLE IF NOT EXISTS forms_html (word text, url text, html text)
""")

c.execute("""
CREATE INDEX IF NOT EXISTS forms_html_words ON forms_html (word)
""")

c.execute("""
CREATE INDEX IF NOT EXISTS words_words ON words (word)
""")

In [3]:
import lxml.html
from xml.etree import ElementTree
import requests
from multiprocessing import Pool
from datetime import datetime

def download(word):
    url = f"https://www.letonika.lv/groups/default.aspx?q={word}&title={word}/0&s=0&g=5&r=1100"
    tree = lxml.html.fromstring(requests.get(url).content)
    return word, url, ElementTree.tostring(tree.xpath("//td[@class='master_vidus']")[0])

pool = Pool(8)
while True:
    c.execute("""
    SELECT DISTINCT words.word FROM words
    LEFT JOIN forms_html
    ON words.word = forms_html.word
    WHERE INSTR(words.word, ' ') = 0 AND forms_html.word IS NULL
    LIMIT 500
    """)
    
    rows = c.fetchall()
    if len(rows) == 0:
        break

    c.executemany(
        "INSERT INTO forms_html VALUES (?, ?, ?)",
        pool.map(download, (row[0] for row in rows))
    )
    conn.commit()
    c.execute("SELECT COUNT(*) FROM forms_html")
    print("Processed", c.fetchall(), datetime.now())

In [4]:
from IPython.core.display import display, HTML
import lxml.html
from xml.etree import ElementTree
import pandas as pd
import numpy as np
import math

c.execute("""
CREATE TABLE IF NOT EXISTS forms
(word text, tags text, col_tags text, type text, form text)
""")

c.execute("""
SELECT forms_html.word, html
FROM forms_html
LEFT JOIN forms
ON forms.word = forms_html.word
WHERE forms.word IS NULL
""")

values = []
for row_index, row in enumerate(c):
    word, html = row
    try:
        tree = lxml.html.fromstring(html.decode("utf-8"))

        results = tree.xpath("//td[@class='Entrytext']")[0]

        entrytext = results.xpath("//span/span[@class='Entrytext']")[0].xpath("string()")
        tables = results.xpath("//span/table")

        print(row_index, word)
        for t in tables:
            df = pd.read_html(ElementTree.tostring(t))[0]
            cols = list(df)
            header_rows = max(
                i for i, ele in enumerate(t.xpath(".//tr")) if ele.attrib.get("bgcolor") == 'lightgrey')+1
            index = list(df[cols[0]])
            for col in cols[1:]:
                col = list(df[col])
                tags = zip(index[:header_rows], col[:header_rows])
                vals = zip(index[header_rows:], col[header_rows:])
                col_tags = " ; ".join(f"{k}: {v}" if str(k) != "nan" and k != v else v for k, v in tags)
                for val in vals:
                    values.append(
                        (word, entrytext, col_tags, val[0], val[1])
                    )
    except Exception as e:
        print("Fail", word, e)

c.executemany(
    "INSERT INTO forms VALUES (?, ?, ?, ?, ?)",
    values
)
conn.commit()
c.execute("SELECT COUNT(*) FROM forms")
print("Saved", c.fetchall())

0 A
1 A
2 A
3 A
Fail AAC list index out of range
Fail AAI list index out of range
Fail Abadāna list index out of range
Fail abdukcija list index out of range
Fail abduktors list index out of range
Fail Abo list index out of range
Fail abonentapakšlīnija list index out of range
11 abpus
Fail abreaģēt list index out of range
Fail abriss list index out of range
Fail abvērs list index out of range
Fail acetātfilma list index out of range
Fail acetātplēve list index out of range
Fail acetātšķiedras list index out of range
Fail acetildihidrokodeīns list index out of range
Fail acetilmetadols list index out of range
Fail aciditāte list index out of range
Fail acidometrs list index out of range
22 acīmredzot
Fail ACP list index out of range
Fail acusakta list index out of range
Fail acvadāma list index out of range
Fail adamantīns list index out of range
Fail adamsīts list index out of range
Fail adatturis list index out of range
Fail ADB list index out of range
Fail Adena list index out of ra

Fail apsardziba list index out of range
Fail aptiekmuzejs list index out of range
Fail apturis list index out of range
Fail aptvarste list index out of range
Fail apudšūnas list index out of range
Fail apvalde list index out of range
Fail apvedrežīms list index out of range
Fail apvērse list index out of range
Fail apzeme list index out of range
Fail apzīmētājkļūme list index out of range
252 apžēliņ!
253 ar
Fail arabiks list index out of range
Fail ARABOSAI list index out of range
Fail aramīds list index out of range
Fail Archie list index out of range
Fail Ardēni list index out of range
259 ardievu!
Fail areometrs list index out of range
Fail argilīts list index out of range
Fail arheoastronomija list index out of range
Fail arheobotānika list index out of range
Fail arheobotāniķis list index out of range
Fail arheogrāfija list index out of range
Fail arheometrija list index out of range
Fail arhīveksemplārs list index out of range
Fail arhīvfonds list index out of range
Fail arhīvis

Fail bagase list index out of range
Fail bagatele list index out of range
Fail bagātināt list index out of range
Fail baikas list index out of range
Fail baikeris list index out of range
500 bail
Fail Baireita list index out of range
Fail bakalaurante list index out of range
Fail bakalaurants list index out of range
Fail bakalaurantūra list index out of range
Fail bakara list index out of range
Fail baks list index out of range
Fail bakss list index out of range
Fail bakterofāgs list index out of range
Fail balādists list index out of range
Fail balbali list index out of range
Fail baletomāns list index out of range
Fail Balhašs list index out of range
Fail balka list index out of range
Fail balsa list index out of range
Fail balss/partija list index out of range
Fail balstieliktnis list index out of range
Fail balstkoks list index out of range
Fail balstplātne list index out of range
Fail balziens list index out of range
Fail baļļuks list index out of range
Fail bandrolēšana list inde

Fail čungurs list index out of range
816 d.
Fail dabasparks list index out of range
Fail dabassaimniecība list index out of range
Fail dacītporfirīts list index out of range
Fail daidžests list index out of range
Fail DAIS list index out of range
Fail dalenis list index out of range
Fail daļapeja list index out of range
Fail dammarsveķi list index out of range
Fail Dāmokls list index out of range
Fail dansings list index out of range
Fail dāņi list index out of range
Fail darbatteice list index out of range
Fail darbdrošība list index out of range
Fail darbgals list index out of range
Fail Dārbi list index out of range
Fail darbībа list index out of range
Fail darbnīcauto list index out of range
Fail darbnīcautomobilis list index out of range
Fail darbzemēšana list index out of range
Fail Darema list index out of range
Fail Dāresalāma list index out of range
Fail darījumdarbība list index out of range
Fail Dārvina list index out of range
Fail DAT list index out of range
Fail datnesis l

Fail dzirksteļkamera list index out of range
Fail dzirksteļsprauga list index out of range
Fail dzīvelīgs list index out of range
Fail dzīvesbiedrs/dzīvesbiedre list index out of range
Fail DZO list index out of range
Fail Džaipūra list index out of range
Fail džanaisms list index out of range
Fail džanduja list index out of range
Fail džaurs list index out of range
Fail džekfrūts list index out of range
Fail Džersisitija list index out of range
Fail džezotēka list index out of range
Fail džigers list index out of range
Fail džingo list index out of range
Fail džingoisms list index out of range
Fail džinsbikses list index out of range
Fail Džungārija list index out of range
Fail e list index out of range
Fail EAP list index out of range
Fail EBCDIC list index out of range
Fail ebulisms list index out of range
Fail eceššļūce list index out of range
Fail EDGE list index out of range
Fail edikts list index out of range
Fail EDMS list index out of range
Fail EDS list index out of range
110

...

Fail tans list index out of range
Fail tanscendents list index out of range
Fail tantalīts list index out of range
Fail tanžīrs list index out of range
Fail tapascirvis list index out of range
Fail tapioka list index out of range
Fail tapnis list index out of range
Fail taranēt list index out of range
Fail tarlatans list index out of range
Fail tartrazīns list index out of range
Fail tartāns list index out of range
Fail tasnieks list index out of range
Fail tauklodīte list index out of range
Fail tauksvītras list index out of range
Fail taukvāre list index out of range
Fail taustcirkulis list index out of range
Fail taustiņslēgs list index out of range
5044 taustoties
Fail tautasmelodija list index out of range
Fail tautasmūzika list index out of range
5047 tavuprāt
5048 taču
5049 te
5050 teciņiem
5051 teciņus
Fail tedeska list index out of range
Fail tedijas list index out of range
Fail tehnofobija list index out of range
Fail tehnogrāfija list index out of range
Fail tehnogēns list i

Fail tupu list index out of range
5284 tur
Fail turas list index out of range
Fail turboagregāts list index out of range
Fail turbīnrats list index out of range
Fail turgors list index out of range
5289 turklāt
5290 turp
5291 turpat
Fail turpinājumapdrošināšana list index out of range
Fail turpinājumresurss list index out of range
Fail turpināmizglītība list index out of range
5295 turpmāk
5296 turpretī
5297 turpu
Fail turētājlīste list index out of range
Fail turētājsliede list index out of range
Fail tusnis list index out of range
Fail tusteps list index out of range
Fail tusēt list index out of range
Fail tusētājs list index out of range
Fail tuvjūras list index out of range
Fail tuvrindsēja list index out of range
Fail tuvskaņas list index out of range
Fail tvaikcaurlaidība list index out of range
Fail tvips list index out of range
Fail tvītot list index out of range
5310 tādējādi
5311 tādēļ
5312 tālab
Fail tālbraucējkapteinis list index out of range
Fail tālerratiņi list index out

Fail violončells list index out of range
Fail virpuļcaurule list index out of range
5591 virs
Fail virsgriezums list index out of range
Fail virsieraksts list index out of range
Fail virsizmaksas list index out of range
Fail virsizmērs list index out of range
Fail virskalandrs list index out of range
Fail virsklājs list index out of range
Fail virskrava list index out of range
Fail virslaide list index out of range
Fail virsmaksa list index out of range
5601 virspus
Fail virspārdots list index out of range
Fail virssalikums list index out of range
Fail virstituls list index out of range
Fail virsvainags list index out of range
Fail virsģērbi list index out of range
Fail virsģērbs list index out of range
Fail virtualitāte list index out of range
Fail virulents list index out of range
Fail virzienstūre list index out of range
Fail virzuļgredzens list index out of range
5612 vis
5613 visapkārt
5614 visbeidzot
5615 viscaur
5616 visgarām
Fail visgudr(el)is list index out of range
Fail visgu

5873 šantažēt
5874 šantažētājs
5875 šantāža
5876 šantāžists
5877 šariats
5878 šarlaks
5879 šarlatāns
5880 šarmants
5881 šarms
Fail šarmēt list index out of range
Fail šarnīr- list index out of range
5884 šarnīra
5885 šarnīrs
5886 šarnīrsavienojums
Fail šarnīruzgrieznis list index out of range
5888 šarnīrveida
Fail šarnīrvira list index out of range
5890 šarpija
5891 šarāde
5892 šaržs
5893 šasija
5894 šatens
5895 šaubas
5896 šaubīgs
5897 šaubīties
5898 šaubīšanās
5899 šaudīgs
5900 šaudīt
5901 šaudīties
5902 šaudīšanās
5903 šaujamais
5904 šaujamierocis
5905 šaujamieroči
5906 šaujamloks
5907 šaujamlūka
5908 šaujampulveris
5909 šaujlauks
5910 šaura
5911 šaurapraide
5912 šauri
Fail šaurierīce list index out of range
5914 šaurjoslas
5915 šaurleņķa
5916 šaurleņķis
5917 šaurs
Fail šaursaslēdze list index out of range
Fail šaursaslēgs list index out of range
Fail šaursliedes list index out of range
5921 šaursliežu
5922 šaurums
5923 šaurība
5924 šausmas
5925 šausminošs
5926 šausmināt
5927 šausmi

Fail šķērstrieciens list index out of range
6367 šķērstroksnis
Fail šķērsvalnītis list index out of range
Fail šķērsvalnīši list index out of range
6370 šķērsvirziens
6371 šķērszarna
6372 šķērszāģis
6373 šķērsām
Fail šķērsēvelmašīna list index out of range
6375 šķērsķieģelis
Fail šķērsšķiedra list index out of range
6377 šķērētājs
Fail šķērīte list index out of range
Fail šķēršošana list index out of range
6380 šķībacainais
6381 šķībacains
6382 šķībi
6383 šķībs
6384 šķībums
6385 šķīdinošs
6386 šķīdinājums
6387 šķīdināt
6388 šķīdinātājs
6389 šķīdināšana
6390 šķīdonis
6391 šķīdoņa
6392 šķīdums
6393 šķīdība
6394 šķīrējtiesa
6395 šķīrējtiesas
6396 šķīrējtiesnesis
6397 šķīst
6398 šķīstamība
6399 šķīstošs
6400 šķīsts
6401 šķīstums
6402 šķīstība
6403 šķīstījošs
6404 šķīstīt
6405 šķīstītava
6406 šķīstītavas
6407 šķīstītavas-
6408 šķīstīties
6409 šķīstīšana
6410 šķīstīšanas-
6411 šķīstīšanās
6412 šķīvis
6413 šķīvji
6414 šķīvjveida
6415 šķīvītis
6416 šķīšana
6417 šķūnis
6418 šķūre
6419 šķūrēt
64

# Cleanup

In [10]:
c.execute("DROP TABLE IF EXISTS lex")

In [11]:
c.execute("""
CREATE TABLE lex AS
SELECT DISTINCT
    CASE
        WHEN INSTR(form, ';') THEN SUBSTR(form, 1, INSTR(form, ';')-1)
        ELSE form
    END AS word,
    CASE
        WHEN INSTR(tags, 'lietvārds') THEN 'lietvārds'
        WHEN INSTR(tags, 'īpašības vārds') THEN 'īpašības vārds'
        WHEN INSTR(tags, 'nenoteiktais vietniekvārds') THEN 'nenoteiktais vietniekvārds'
        WHEN INSTR(tags, 'darbības vārds') THEN 'darbības vārds'
        WHEN INSTR(tags, 'apstākļa vārds') THEN 'apstākļa vārds'
        WHEN INSTR(tags, 'pamata skaitļa vārds') THEN 'pamata skaitļa vārds'
        WHEN INSTR(tags, 'kārtas skaitļa vārds') THEN 'kārtas skaitļa vārds'
        WHEN INSTR(tags, 'saiklis') THEN 'saiklis'
        WHEN INSTR(tags, 'izsauksmes vārds') THEN 'izsauksmes vārds'
        WHEN INSTR(tags, 'personas vietniekvārds') THEN 'personas vietniekvārds'
        WHEN INSTR(tags, 'izsauksmes vārds') THEN 'izsauksmes vārds'
        WHEN INSTR(tags, 'saīsinājums') THEN 'saīsinājums'
        WHEN INSTR(tags, 'prievārds') THEN 'prievārds'
        
        WHEN INSTR(tags, 'vispārināmais vietniekvārds') THEN 'vispārināmais vietniekvārds'
        WHEN INSTR(tags, 'norādāmais vietniekvārds') THEN 'norādāmais vietniekvārds'
        WHEN INSTR(tags, 'noliedzamais vietniekvārds') THEN 'noliedzamais vietniekvārds'
        WHEN INSTR(tags, 'jautājamais vietniekvārds') THEN 'jautājamais vietniekvārds'
        WHEN INSTR(tags, 'noteicamais vietniekvārds') THEN 'noteicamais vietniekvārds'
        WHEN INSTR(tags, 'attieksmes vietniekvārds') THEN 'attieksmes vietniekvārds'
        WHEN INSTR(tags, 'piederības vietniekvārds') THEN 'piederības vietniekvārds'
        
        WHEN INSTR(tags, 'lokāms ciešamās kārtas Tagadnes divdabis') THEN 'lokāms ciešamās kārtas Tagadnes divdabis'
        WHEN INSTR(tags, 'lokāms darāmās kārtas Tagadnes divdabis') THEN 'lokāms darāmās kārtas Tagadnes divdabis'
        WHEN INSTR(tags, 'lokāms ciešamās kārtas Pagātnes divdabis') THEN 'lokāms ciešamās kārtas Pagātnes divdabis'
        WHEN INSTR(tags, 'lokāms darāmās kārtas Pagātnes divdabis') THEN 'lokāms darāmās kārtas Pagātnes divdabis'

        ELSE ''
    END AS type,
    CASE
        WHEN INSTR(col_tags, 'Vienskaitlis') THEN 'vienskaitlis'
        WHEN INSTR(col_tags, 'Daudzskaitlis') THEN 'daudzskaitlis'

        WHEN type = 'Es' OR type = 'Tu' OR type = 'Viņš; viņa' THEN 'vienskaitlis'
        WHEN type = 'Mēs' OR type = 'Jūs' OR type = 'Viņi; viņas' THEN 'daudzskaitlis'
        ELSE ''
    END AS value,
    CASE
        WHEN INSTR(col_tags, 'Vīriešu dzimte') THEN 'vīriešu dzimte'
        WHEN INSTR(col_tags, 'Sieviešu dzimte') THEN 'sieviešu dzimte'
        ELSE ''
    END AS gender,
    CASE
        WHEN INSTR(col_tags, 'Tagadne') THEN 'tagadne'
        WHEN INSTR(col_tags, 'Pagātne') THEN 'pagātne'
        WHEN INSTR(col_tags, 'Nākotne') THEN 'nākotne'
        
        WHEN form = word  AND type = 'darbības vārds' THEN 'infinitive'
        ELSE ''
    END AS tense,
    CASE
        WHEN type = 'Es' OR type = 'Mēs' THEN '1. persona'
        WHEN type = 'Tu' OR type = 'Jūs' THEN '2. persona'
        WHEN type = 'Viņš; viņa' OR type = 'Viņi; viņas' THEN '3. persona'
        ELSE ''
    END AS person,
    CASE
        WHEN INSTR(tags, 'nenoteiksme') THEN 'nenoteiksme'
        WHEN INSTR(tags, 'pavēles izteiksme') THEN 'pavēles izteiksme'
        WHEN INSTR(tags, 'īstenības izteiksme') THEN 'īstenības izteiksme'
        WHEN INSTR(tags, 'vajadzības izteiksme') THEN 'vajadzības izteiksme'
        ELSE ''
    END AS mood,
    CASE
        WHEN INSTR(col_tags, 'Pārākā pakāpe') THEN 'pārākā pakāpe'
        WHEN INSTR(col_tags, 'Pamata pakāpe') THEN 'pamata pakāpe'
        WHEN INSTR(col_tags, 'Vispārākā pakāpe') THEN 'vispārākā pakāpe'
        ELSE ''
    END AS degree,
    CASE
        WHEN type = 'Nominatīvs' THEN 'nominatīvs'
        WHEN type = 'Akuzatīvs' THEN 'akuzatīvs'
        WHEN type = 'Ģenitīvs' THEN 'ģenitīvs'
        WHEN type = 'Datīvs' THEN 'datīvs'
        WHEN type = 'Lokatīvs' THEN 'lokatīvs'
        WHEN type = 'Vokatīvs' THEN 'vokatīvs'
        ELSE ''
    END AS nouncase,
    CASE
        WHEN INSTR(col_tags, 'noteiktā galotne') THEN 'noteiktā galotne'
        WHEN INSTR(col_tags, 'nenoteiktā galotne') THEN 'nenoteiktā galotne'
        ELSE ''
    END AS definite
FROM forms
""")
conn.commit()